In [45]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import sys
sys.path.append('../scripts/')
from data.dataset_binary import GetData
from train.train_binary import train_binary_fn
from model.c3d import C3D
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")
from functools import partial
from tqdm import tqdm
import torch
from train.metrics_binary import recall_binary, precision_binary

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
obj = GetData(label_path='./train.csv')
obj.split_datasets() 
obj.perp_dataset()
obj.create_dataset()
obj.create_loaders(batch_size=64)

In [3]:
train_loader, val_loader, test_loader = obj.return_loaders()

In [4]:
wandb_dict = {
    "epochs": 30,
    "batch_size": 64,
    "learning_rate": 0.01,
    "model": "C3D",
    "dataset": "Binary",
    "optimizer": "SGD",
}

In [5]:
model = C3D()

In [6]:
partial_binary_fn = partial(train_binary_fn, model = model, train_loader = train_loader, val_loader = val_loader, wandb_dict = wandb_dict)

In [7]:
partial_binary_fn(epochs = 30, save_weights_path='../../weights/C3D_v1/', load_model = '../../weights/C3D/model_29.pth' load_model=None)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: yashchks87. Use `wandb login --relogin` to force relogin


100%|██████████| 5/5 [00:04<00:00,  1.22batch/s, loss=3.43, prec=4.02, rec=4.62]   


Epoch 0/30 | Train Loss: 0.6904669582843781 | Val Loss: 0.6852413654327393
Epoch 0/30 | Train Prec: 0.5428289100527763 | Val Prec: 0.8048377156257629
Epoch 0/30 | Train Rec: 0.9930228471755982 | Val Rec: 0.923006558418274
Weight saved for epoch: 0


100%|██████████| 5/5 [00:03<00:00,  1.41batch/s, loss=3.27, prec=4.78, rec=3.02]


...

Epoch 2/30 | Train Loss: 0.5620946601033211 | Val Loss: 0.4322411775588989
Epoch 2/30 | Train Prec: 0.9487437576055526 | Val Prec: 0.9515393733978271
Epoch 2/30 | Train Rec: 0.597312219440937 | Val Rec: 0.6509671926498413
Weight saved for epoch: 2


100%|██████████| 5/5 [00:04<00:00,  1.14batch/s, loss=1.41, prec=4.7, rec=4.39]  


Epoch 3/30 | Train Loss: 0.3312382273375988 | Val Loss: 0.2810929179191589
Epoch 3/30 | Train Prec: 0.9248945206403733 | Val Prec: 0.9393027186393738
Epoch 3/30 | Train Rec: 0.7868835210800171 | Val Rec: 0.8770614862442017
Weight saved for epoch: 3


  0%|          | 0/20 [00:01<?, ?batch/s]


KeyboardInterrupt: 

### Evaluate model

In [79]:
model = C3D().to('cuda')
model.load_state_dict(torch.load('../../weights/C3D/model_28.pth')['model_state_dict'], strict=False)

<All keys matched successfully>

In [80]:
# Get predictions
predictions = []
labels = []
# temp = None
for img, label in tqdm(test_loader):
    img = img.to('cuda')
    preds = model.predictions(img)
    temp = preds.detach().cpu().numpy().tolist()
    predictions.extend(temp)
    labels.extend(label.numpy().tolist())

100%|██████████| 7/7 [00:04<00:00,  1.74it/s]


In [81]:
p = torch.Tensor(np.array(predictions).flatten())
l = torch.Tensor(np.array(labels).flatten())

In [82]:
p_sigmoid = torch.where(torch.nn.functional.sigmoid(p) > 0.5, 1, 0)

In [84]:
recall_binary(p, l)

tensor(0.9841)

In [85]:
precision_binary(p, l)

tensor(0.9394)